<a href="https://colab.research.google.com/github/Nathan-Roll1/ProsodPy/blob/main/Tutorials/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount necessary data files from Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from tqdm import tqdm
import shutil

Mounted at /content/drive


In [ ]:
# Use functions for preprocessing
try:
  shutil.rmtree('ProsodPy')
except:
  pass
!git clone https://github.com/Nathan-Roll1/ProsodPy

from ProsodPy.ProsodPy import *

# plotting options intended for 20x3 dimensions
plt.rcParams["figure.figsize"] = (20,3)

Cloning into 'ProsodPy'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 180 (delta 0), reused 0 (delta 0), pack-reused 176
Receiving objects: 100% (180/180), 22.99 MiB | 18.39 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [ ]:
# path to resampled discourse audio
init_path = 'drive/Shareddrives/Ling Thesis/resampled/'

# get name of each file
fs = os.listdir(init_path)

# merge names with path to generate full path
fs = [init_path + x for x in fs]

In [ ]:
# path to transcripts
text_path = 'drive/Shareddrives/Rezonator Data/SBCSAE/'

# for every audio file...
for n, f in enumerate(tqdm(fs)):

  # get matching transcript file
  tt = f.split('/')[-1].split('_')[0]
  txt = text_path + tt +'.cha'

  # identify boundary timestamps
  boundaries = get_boundaries(txt)

  # adjust for 8k sample rate
  boundaries = boundaries*8000 

  # load audio file
  with open(f, 'rb') as handle:
    audio_array = pickle.load(handle)
    
  # generate features and labels
  boundary_mfcc, labels, __ = MFCC_preprocess(audio_array, boundaries, hop_length=16, n_mfcc = 15, n_fft=743, n_frames = 1024, normalize=True)

  # negative sample false labels to equalize classes
  
  size = sum(labels==1)
  indicies = list(np.random.choice(np.where(labels == 0)[0], int(size)))
  indicies += list(np.where(labels == 1)[0])

  # apply selected indices to featuresets and labels
  labels = labels[indicies] 
  boundary_mfcc = boundary_mfcc[indicies]

  # save features
  with open(f'drive/Shareddrives/Ling Thesis/PREPROCESSED/features/{tt}_hop16_coef15_normalized.pickle', 'wb') as handle:
    pickle.dump(boundary_mfcc, handle, protocol=pickle.HIGHEST_PROTOCOL)

  # save labels
  with open(f'drive/Shareddrives/Ling Thesis/PREPROCESSED/labels/{tt}_hop16_coef15_normalized.pickle', 'wb') as handle:
    pickle.dump(labels, handle, protocol=pickle.HIGHEST_PROTOCOL)